# Homework 4

## FINM 37500: Fixed Income Derivatives

### Mark Hendricks

#### Winter 2025

***

### Data

The file `data/ratetree_data_2025-01-31.xlsx` has a binomial tree of interest rates fit to...
* discount curves from `cap_curves_2025-01-31.xlsx`
* implied vols from `cap_curves_2025-01-31.xlsx`

Note the following...
* Suppose the present date is `2025-01-31`.
* The rates are continuously compounded.
* The rates are for the following quarter. So teh rate at $t=0$ is the continuously compounded rate for the interval $t=0$ to $t=.25$.

Take this binomial tree as given; there is no need to fit it yourself.

In [4]:
import pandas as pd
import numpy as np

DATE = '2025-01-31'
FILEIN = f'../data/ratetree_data_{DATE}.xlsx'
sheet_tree = 'rate tree'

ratetree = pd.read_excel(FILEIN, sheet_name=sheet_tree).set_index('state')
ratetree.columns.name = 'time'

ratetree.style.format('{:.1%}',na_rep='').format_index('{:.2f}',axis=1)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,4.2%,4.3%,4.4%,4.8%,5.1%,6.4%,7.6%,9.1%,10.1%,11.8%,13.5%,15.3%,16.9%,19.2%,22.7%,25.9%,28.2%,30.6%,34.7%,40.3%
1,,3.9%,4.0%,4.2%,4.4%,5.2%,6.0%,7.0%,7.8%,9.0%,10.4%,11.8%,13.0%,14.8%,17.4%,19.9%,21.7%,23.7%,27.0%,31.2%
2,,,3.6%,3.7%,3.8%,4.2%,4.7%,5.4%,6.0%,7.0%,8.0%,9.1%,10.1%,11.4%,13.4%,15.3%,16.7%,18.4%,20.9%,24.2%
3,,,,3.3%,3.2%,3.3%,3.7%,4.2%,4.6%,5.3%,6.2%,7.0%,7.8%,8.8%,10.3%,11.8%,12.9%,14.2%,16.3%,18.8%
4,,,,,2.8%,2.7%,2.9%,3.3%,3.6%,4.1%,4.7%,5.4%,6.0%,6.8%,7.9%,9.1%,10.0%,11.0%,12.6%,14.6%
5,,,,,,2.2%,2.3%,2.5%,2.7%,3.2%,3.6%,4.2%,4.6%,5.2%,6.1%,7.0%,7.7%,8.6%,9.8%,11.3%
6,,,,,,,1.8%,2.0%,2.1%,2.4%,2.8%,3.2%,3.6%,4.0%,4.7%,5.4%,5.9%,6.6%,7.6%,8.8%
7,,,,,,,,1.5%,1.6%,1.9%,2.2%,2.5%,2.7%,3.1%,3.6%,4.1%,4.6%,5.1%,5.9%,6.8%
8,,,,,,,,,1.3%,1.4%,1.7%,1.9%,2.1%,2.4%,2.8%,3.2%,3.5%,4.0%,4.6%,5.3%


***

# 1. Binomial Tree Pricing - Bond

### The Bond

Consider a vanilla (non-callable) bond with the following parameters...
* `T=5`
* coupon rate is `4.41%`
* coupons are semiannual

Note that this is essentially the hypothetical bond priced in HW 1.

### 1.1

Create and display a tree of cashflows from the bond, corresponding to each node of the tree (state and time) seen in the interest rate tree.

Note that the cashflows do not depend on the interest rates. Thus, report the cashflows at the time (in the column) they are actually paid out. The final payoff (face plus coupon) occurs at $T$, which is beyond the interest rate tree. You are welcome to add a column for $T$ or to consider this payoff separately and leave it out of the tree.

In [3]:
cf_df = pd.DataFrame({0: [0]})

for i in range(1, 21):
    if i == 20:
        payout = 100 + 2.205
    elif i % 2 == 0:
        payout = 2.205
    else:
        payout = 0
    cf_df[i / 4] = [payout]

cf_df

,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75,5.00
0,0,0,2.205,0,2.205,0,2.205,0,2.205,0,...,0,2.205,0,2.205,0,2.205,0,2.205,0,102.205


### 1.2.

Create and display a tree of values of the bond. Do this for the quotes as
* clean quotes
* dirty quotes

Given the semiannual coupons and quarterly tree steps, the clean and dirty will coincide at $t=0, .5, 1,...$.

Do the valuation by...
* setting the value at $T$ as the face plus final coupon.
* discounting this back through time, using the (continuously-compounded) interest rate.
* recall that the tree is constructed such that the probability of moving "up" or "down" is 50%.

In [25]:
def price_vanilla_bond(ratetree, cpn_rate, face=100, freq=2, clean=True):
    cpn = cpn_rate * face / freq
    disc_tree = np.exp(ratetree / 4) # discount tree
    val_tree = ratetree.copy()
    

    for i in range(19, -1, -1):
        time = i / 4
        if i == 19:
            val_tree[time] = (face + cpn) / disc_tree[time]
            continue
        
        avg_series = val_tree[time + 0.25].rolling(window=2).mean()
        avg_series = avg_series.shift(-1)
        
        if i % 2 == 0:
            val_tree[time] = cpn + avg_series / disc_tree[time]
        else:
            val_tree[time] = avg_series / disc_tree[time]

        
    if not clean: # add accrued interest
        for t in range(1, 21, 2):
            ti = t / 4
            val_tree[ti] += cpn / 2
    
    return val_tree
    


In [ ]:
# clean price tree
price_vanilla_bond(ratetree, 0.0441)

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3,3.25,3.5,3.75,4,4.25,4.5,4.75
state,,,,,,,,,,,,,,,,,,,,
0,104.074401,101.020606,100.038066,96.674771,95.412415,91.804347,90.561114,87.209059,86.276634,83.234494,82.683237,80.182270,80.271327,78.422727,79.356835,78.871143,81.619125,83.017188,87.901687,92.411267
1,NaN,104.874751,104.198996,101.179152,100.262969,97.006495,96.027611,92.901158,92.134736,89.229230,88.757972,86.297496,86.364216,84.453333,85.218808,84.423561,86.664663,87.393012,91.350206,94.526329
2,NaN,NaN,107.609029,104.868428,104.234514,101.267177,100.510592,97.577717,96.958541,94.177536,93.782819,91.361871,91.411396,89.445063,90.062346,88.994402,90.795758,90.948229,94.121954,96.199500
3,NaN,NaN,NaN,107.861708,107.450967,104.713944,104.138429,101.366170,100.872595,98.200304,97.874791,95.490134,95.526543,93.512637,94.004142,92.704542,94.136113,93.806834,96.332291,97.517148
4,NaN,NaN,NaN,NaN,110.034062,107.475596,107.043995,104.401717,104.012321,101.432140,101.166870,98.814212,98.840717,96.787106,97.174574,95.683329,96.811009,96.086825,98.084197,98.551194
5,NaN,NaN,NaN,NaN,NaN,109.671862,109.352437,106.813373,106.508629,104.004803,103.790593,101.465365,101.484465,99.398401,99.701469,98.054765,98.936965,97.893949,99.466184,99.360480
6,NaN,NaN,NaN,NaN,NaN,NaN,111.175060,108.716765,108.479758,106.038212,105.866449,103.564265,103.577880,101.465706,101.701306,99.930349,100.616810,99.319321,100.552357,99.992531
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110.211354,110.027926,107.636569,107.499592,105.216479,105.226070,103.093144,103.275412,101.406252,101.938170,100.439347,101.403601,100.485356
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.238893,108.887614,108.778841,106.511335,106.518000,104.368733,104.509208,102.563094,102.973917,101.316862,102.069250,100.869139


In [28]:
# dirty price tree
price_vanilla_bond(ratetree, 0.0441, clean=False)

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3,3.25,3.5,3.75,4,4.25,4.5,4.75
state,,,,,,,,,,,,,,,,,,,,
0,104.074401,102.123106,100.038066,97.777271,95.412415,92.906847,90.561114,88.311559,86.276634,84.336994,82.683237,81.284770,80.271327,79.525227,79.356835,79.973643,81.619125,84.119688,87.901687,93.513767
1,NaN,105.977251,104.198996,102.281652,100.262969,98.108995,96.027611,94.003658,92.134736,90.331730,88.757972,87.399996,86.364216,85.555833,85.218808,85.526061,86.664663,88.495512,91.350206,95.628829
2,NaN,NaN,107.609029,105.970928,104.234514,102.369677,100.510592,98.680217,96.958541,95.280036,93.782819,92.464371,91.411396,90.547563,90.062346,90.096902,90.795758,92.050729,94.121954,97.302000
3,NaN,NaN,NaN,108.964208,107.450967,105.816444,104.138429,102.468670,100.872595,99.302804,97.874791,96.592634,95.526543,94.615137,94.004142,93.807042,94.136113,94.909334,96.332291,98.619648
4,NaN,NaN,NaN,NaN,110.034062,108.578096,107.043995,105.504217,104.012321,102.534640,101.166870,99.916712,98.840717,97.889606,97.174574,96.785829,96.811009,97.189325,98.084197,99.653694
5,NaN,NaN,NaN,NaN,NaN,110.774362,109.352437,107.915873,106.508629,105.107303,103.790593,102.567865,101.484465,100.500901,99.701469,99.157265,98.936965,98.996449,99.466184,100.462980
6,NaN,NaN,NaN,NaN,NaN,NaN,111.175060,109.819265,108.479758,107.140712,105.866449,104.666765,103.577880,102.568206,101.701306,101.032849,100.616810,100.421821,100.552357,101.095031
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.313854,110.027926,108.739069,107.499592,106.318979,105.226070,104.195644,103.275412,102.508752,101.938170,101.541847,101.403601,101.587856
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.238893,109.990114,108.778841,107.613835,106.518000,105.471233,104.509208,103.665594,102.973917,102.419362,102.069250,101.971639


### 1.3.

The binomial-estimated price of the bond is the initial node of the value tree.

Report this along with the price of the bond you would get from the usual simple formula for such a bond. 
* Consider pricing it with the $T$ interval swap rate (used similar to a ytm) from the file `cap_curves_2025-01-31.xlsx`.
* If you do this, recall that the swap rate given in that file is quarterly-compounded, so you would need to convert it to semiannual compounding before plugging it into the usual closed-form ytm-pricing formula.

In [34]:
cap_path = "../data/cap_curves_2025-01-31.xlsx"
swap_df = pd.read_excel(cap_path).set_index('tenor')

In [37]:
swap_df['discount rates'] = np.exp(- swap_df['swap rates'] / 4).cumprod()

cpn = 4.41 / 2
swap_price = cpn
for t in range(1, 11):
    swap_price += cpn * swap_df['discount rates'].loc[t / 2]
    if t == 10:
        swap_price += 100 * swap_df['discount rates'].loc[t / 2]

print(f"The price based upon swaps is {round(swap_price, 2)}")

The price based upon swaps is 103.78


### Note:

An easy check on your code is whether it will correctly price a zero-coupon bond at a price that matches the "discounts" in the `cap_curves` data file.

***

# 2. Pricing the Callable - European

### 2.1.

Calculate and display value tree of a European-style call option on the bond analyzed in part `1`.
* `$T_o = 3$`. That is, the time-to-expiration is 3 years.
* `$K=100$`. That is, the strike is 100. This is a clean strike, meaning exercise requires paying the strike plus any accrued interest.

Do so by 
* setting the value at the time of expiration, using the value of the bond for each node at that time.
* discounting this back through time, using the (continuously-compounded) interest rate.
* recall that the tree is constructed such that the probability of moving "up" or "down" is 50%.

Note that...
* the tree of call values will not be the same size as the tree of bond values. The former goes only to $T_o=3$.

In [74]:
def price_bond_call(ratetree, cpn, expiry, strike, freq=2, eur=True):
    disc_tree = np.exp(ratetree / 4)
    clean_tree = price_vanilla_bond(ratetree, cpn, freq=2)
    option_tree = clean_tree.copy()
    option_tree = option_tree[np.arange(0, 3 + 0.25, 0.25)]

    if eur:
        option_tree[expiry] = np.maximum(0, option_tree[expiry] - strike)

        for t in np.arange(expiry - 0.25, -0.25, -0.25):
            avg_series = option_tree[t + 0.25].rolling(window=2).mean()
            avg_series = avg_series.shift(-1)
            option_tree[t] = avg_series / disc_tree[t]

    else:
        option_tree = np.maximum(0, option_tree - strike)

        for t in np.arange(expiry - 0.25, -0.25, -0.25):
            avg_series = option_tree[t + 0.25].rolling(window=2).mean()
            avg_series = avg_series.shift(-1)

            option_tree[t] = np.maximum(avg_series / disc_tree[t], option_tree[t])

    return option_tree
        

In [75]:
price_bond_call(ratetree, 0.0441, 3, 100)

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3
state,,,,,,,,,,,,,
0,3.118997,2.493648,1.881670,1.312683,0.820569,0.435626,0.177591,0.042461,0.000000,0.000000,0.000000,0.000000,0.000000
1,NaN,3.810374,3.159831,2.492736,1.836379,1.226605,0.707802,0.319550,0.086867,0.000000,0.000000,0.000000,0.000000
2,NaN,NaN,4.535708,3.890733,3.201795,2.486561,1.777338,1.117432,0.563553,0.177161,0.000000,0.000000,0.000000
3,NaN,NaN,NaN,5.263394,4.651966,3.977347,3.247710,2.479418,1.701952,0.967019,0.360534,0.000000,0.000000
4,NaN,NaN,NaN,NaN,5.960785,5.401621,4.773626,4.076571,3.309536,2.476510,1.599512,0.732241,0.000000
5,NaN,NaN,NaN,NaN,NaN,6.602280,6.102253,5.540670,4.910669,4.201802,3.404615,2.504861,1.484465
6,NaN,NaN,NaN,NaN,NaN,NaN,7.173571,6.734186,6.241302,5.687132,5.065545,4.366654,3.577880
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.677996,7.293606,6.861557,6.377885,5.835675,5.226070
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.121192,7.785069,7.409315,6.989063,6.518000


### 2.2.

Show the value tree of the callable bond by subtracting the call value tree from the (subset $t\le T_o$ of the) bond value tree (calculated in part `1`.) Do this for both
* clean
* dirty

In [76]:
call_tree = price_bond_call(ratetree, 0.0441, 3, 100)
clean_tree = price_vanilla_bond(ratetree, 0.0441)
dirty_tree = price_vanilla_bond(ratetree, 0.0441, clean=False)

clean_callable = clean_tree[np.arange(0, 3.25, 0.25)] - call_tree
dirty_callable = clean_tree[np.arange(0, 3.25, 0.25)] - call_tree

In [77]:
# clean callable price tree
clean_callable

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3
state,,,,,,,,,,,,,
0,100.955404,98.526958,98.156396,95.362088,94.591846,91.368721,90.383523,87.166598,86.276634,83.234494,82.683237,80.182270,80.271327
1,NaN,101.064377,101.039166,98.686416,98.426590,95.779890,95.319809,92.581608,92.047869,89.229230,88.757972,86.297496,86.364216
2,NaN,NaN,103.073320,100.977696,101.032718,98.780616,98.733254,96.460285,96.394988,94.000375,93.782819,91.361871,91.411396
3,NaN,NaN,NaN,102.598313,102.799001,100.736597,100.890719,98.886752,99.170643,97.233284,97.514257,95.490134,95.526543
4,NaN,NaN,NaN,NaN,104.073277,102.073975,102.270369,100.325146,100.702784,98.955629,99.567358,98.081971,98.840717
5,NaN,NaN,NaN,NaN,NaN,103.069582,103.250184,101.272704,101.597960,99.803002,100.385979,98.960505,100.000000
6,NaN,NaN,NaN,NaN,NaN,NaN,104.001489,101.982579,102.238456,100.351081,100.800904,99.197611,100.000000
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.533358,102.734320,100.775012,101.121707,99.380803,100.000000
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.117701,101.102545,101.369526,99.522272,100.000000


In [78]:
# dirty callable price tree
dirty_callable

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3
state,,,,,,,,,,,,,
0,100.955404,98.526958,98.156396,95.362088,94.591846,91.368721,90.383523,87.166598,86.276634,83.234494,82.683237,80.182270,80.271327
1,NaN,101.064377,101.039166,98.686416,98.426590,95.779890,95.319809,92.581608,92.047869,89.229230,88.757972,86.297496,86.364216
2,NaN,NaN,103.073320,100.977696,101.032718,98.780616,98.733254,96.460285,96.394988,94.000375,93.782819,91.361871,91.411396
3,NaN,NaN,NaN,102.598313,102.799001,100.736597,100.890719,98.886752,99.170643,97.233284,97.514257,95.490134,95.526543
4,NaN,NaN,NaN,NaN,104.073277,102.073975,102.270369,100.325146,100.702784,98.955629,99.567358,98.081971,98.840717
5,NaN,NaN,NaN,NaN,NaN,103.069582,103.250184,101.272704,101.597960,99.803002,100.385979,98.960505,100.000000
6,NaN,NaN,NaN,NaN,NaN,NaN,104.001489,101.982579,102.238456,100.351081,100.800904,99.197611,100.000000
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.533358,102.734320,100.775012,101.121707,99.380803,100.000000
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.117701,101.102545,101.369526,99.522272,100.000000


### 2.3.

Report the initial node value of the call option and of the callable bond.

In a table, compare these to what you got in HW 1 as the value of the embedded call and the value of the callable bond.
* In `HW 1`, we were valuing from a date nearly two weeks later, `2025-02-13`. This difference in the timing means we wouldn't expect the values to match exactly, even if the methods were entirely consistent.

***

In [79]:
callable_init = clean_callable[0].loc[0]
call_init = call_tree[0].loc[0]

print(f"Callable bond value: {callable_init}")
print(f"Call value: {call_init}")


Callable bond value: 100.95540375772337
Call value: 3.118996821982118


**Answer**

The call value that we calculated is 2.90. The callable was 100.9 for market value. The calculated value is 98.67. Both of these are similar.

# 3. Pricing the Callable - American

### 3.1.

Re-do part `2.`, but this time, make the option a **American** style. That is, allow it to be exercised at any node.
* Report the tree of callable-bond values.
* How does this compare to the European-style?

#### Note
To do this valuation, go through the procedure in `2.1.`, but at each node, compare the value for the call with the value of the payoff function based on the vanilla bond's value at that node. Take the maximum of the two. If you code this carefully, you can simply add a line of code to what you did in `2.1`.

In [82]:
amer_call_tree = price_bond_call(ratetree, 0.0441, 3, 100, eur=False)

In [83]:
amer_call_tree

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3
state,,,,,,,,,,,,,
0,4.60526,3.341665,2.315189,1.507514,0.884822,0.446912,0.177591,0.042461,0.000000,0.000000,0.000000,0.000000,0.000000
1,NaN,5.966348,4.440776,3.174638,2.166475,1.345477,0.730739,0.319550,0.086867,0.000000,0.000000,0.000000,0.000000
2,NaN,NaN,7.609029,5.796589,4.249921,3.035145,1.995240,1.163998,0.563553,0.177161,0.000000,0.000000,0.000000
3,NaN,NaN,NaN,8.671701,7.450967,5.544759,4.138429,2.873827,1.796362,0.967019,0.360534,0.000000,0.000000
4,NaN,NaN,NaN,NaN,10.034062,8.143463,7.043995,5.217558,4.012321,2.667529,1.599512,0.732241,0.000000
5,NaN,NaN,NaN,NaN,NaN,10.208653,9.352437,7.446728,6.508629,4.790580,3.790593,2.504861,1.484465
6,NaN,NaN,NaN,NaN,NaN,NaN,11.175060,9.208351,8.479758,6.642627,5.866449,4.366654,3.577880
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.592844,10.027926,8.101384,7.499592,5.835675,5.226070
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.238893,9.245014,8.778841,6.989063,6.518000


### 3.2.

In which nodes will the American-style callable bond be exercised?

In [88]:
price_diff = clean_tree - amer_call_tree

proc_df = np.where(pd.isna(price_diff), np.nan, price_diff == 100)
proc_df = pd.DataFrame(proc_df, index=clean_tree.index, columns=clean_tree.columns)

def highlight_cells(val):
    if pd.isna(val):
        return ''
    elif val:
        return 'background-color: green'
    else:
        return 'background-color: red'

styled_proc_df = proc_df.style.applymap(highlight_cells)
styled_proc_df

C:\Users\xuv14\AppData\Local\Temp\ipykernel_48368\789674499.py:14: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_proc_df = proc_df.style.applymap(highlight_cells)


time,0,0.250000,0.500000,0.750000,1,1.250000,1.500000,1.750000,2,2.250000,2.500000,2.750000,3,3.250000,3.500000,3.750000,4,4.250000,4.500000,4.750000
state,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan,nan,nan
1,nan,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan,nan,nan
2,nan,nan,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan,nan,nan
3,nan,nan,nan,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan,nan,nan
4,nan,nan,nan,nan,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan,nan,nan
5,nan,nan,nan,nan,nan,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,nan,nan,nan,nan,nan,nan,nan
6,nan,nan,nan,nan,nan,nan,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,nan,nan,nan,nan,nan,nan,nan
7,nan,nan,nan,nan,nan,nan,nan,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,nan,nan,nan,nan,nan,nan,nan
8,nan,nan,nan,nan,nan,nan,nan,nan,1.000000,0.000000,1.000000,0.000000,1.000000,nan,nan,nan,nan,nan,nan,nan


***

# 4. Pricing the Callable - Bermudan

#### This Section is NOT REQUIRED and NOT EXPECTED
Still, it is not much additional work, and some of you may find it interesting. It also illustrates the power of binomial trees in how easily they handle the Bermudan style. 

### 4.1.

Re-do part `3`, but this time with **Bermudan** style exercise. 
* This corresponds to the Freddie Mac bond in `HW 1`.
* Note that the option value tree will now go all the way to $T$.

As a reminder, the Bermudan style can be exercised as early as $T_o$ all the way to $T$. It can only be exercised on specific dates at 3-month intervals, but in our quarterly-spaced tree, this means every node from $T_o$ onward.

### 4.2.

Compare the valuation to the market quote in `HW 1`.

***